In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [3]:
Customer = namedtuple('Customer', 'name fidelity')
Customer

__main__.Customer

In [18]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return discount
        
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [16]:
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount."""
        

class FidelityPromotion(Promotion):
    """5% discount for customer with 1000 or more fidelity points."""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    

class BulkItemPromo(Promotion):
    """10% discount for each lineitem with 20 or more units."""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >=20:
                discount += item.total() * .1
                
        return discount
    
class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more discutinct items."""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.card}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [17]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [19]:
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5), 
       LineItem('watermellon', 5, 5.0)
       ]

In [23]:
print(Order(joe, cart, FidelityPromotion()))
print(Order(ann, cart, FidelityPromotion()))

<Order total: 42.00 due: 0.00>
<Order total: 42.00 due: 2.10>


## Let's do it with less code, with functions as objects

In [54]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total()-discount
        
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [55]:
def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [56]:
def bulk_order_promo(order):
    """10% discount for each lineitem with 20 or more units."""
    
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() * .1

    return discount
    
def large_order_promo(order):
    """7% discount for orders with 10 or more discutinct items."""
       
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [57]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

## Best discount meta strategy

In [58]:
promos = [fidelity_promo, bulk_order_promo, large_order_promo]
def best_promo(order):
    return max(promo(order) for promo in promos)

In [60]:
# but, this means we may not forget any promo strategy in the promos list ... not great

In [59]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

# Even better discount strategy finding all discounts automatically

In [62]:
globals()

AttributeError: 'function' object has no attribute 'discount'